# Understanding the Resps times, and getting volatilities for free

In this notebook we investigate the different meanings of the `resp` variables. As specified in the dataset description:

>In the training set, train.csv, you are provided [...] resp_{1,2,3,4} values that represent returns over different time horizons.

Our goal is to find these time horizons. While doing so, we will also see that we manage to estimate the volatilities of each trade.

# Estimating trade volatility and return time horizons by maximum likelyhood

If for every given trade the time horizons for returns `resp_1` to `resp_4` are fixed, we can estimate how far in time `resp_1` to `resp_4` are, together with the variance of the returns of the trade. This in turn could be useful for further analysis.

Let's consider `resp_1`, `resp_2`, `resp_3`, `resp_4` at row $i$ and call them $r_{1}$, $r_{2}$, $r_{3}$, $r_{4}$ for ease of notation. If the $n$ index is present in the notation, as  $r_{nj}$, it means that I'm talking specifically about row $n$.

The`resp_j` values represent the increment in price from the time of the trade ($T_0$), to some horizon $T_j$  $$r_{j} = \frac{S_{T_j}}{S_{T_0}} - 1$$
Without imposing too heavy assumptions, we can model the prices process in time as the exponential of a random walk, such that at time $t$, the price $S_t$ equals $S_0 e^{\sigma W_t}$, where $W_t$ is Brownian motion (gaussian random walk, markov process where $W_{t_2} - W_{t_1} \sim \mathcal{N}(0, t_2 - t_1)$) and $\sigma$ is the volatility of the stock. The data we have are the `resp_j` $r_j$, from which we can recover the value of the underlying random walk at times $T_1, T_2, T_3, T_4$, since
$$
r_j = \frac{S_{T_j}}{S_{T_0}} - 1 = \frac{S_{T_0} e^{\sigma W_{T_j}}}{S_{T_0}} - 1 = e^{\sigma W_{T_j}} - 1.
$$
Therefore, by adding 1 to $r_j$ and taking the logarithm we recover $\sigma W_{T_j}$ 

Since $\sigma W_{T_j}$ and $\sigma W_{T_k}$ for $j \neq k$ are correlated, and the amount of correlation allows us to measure the relative positions in time of $T_j$ and $T_k$. To find the time values we take the first differences of the five values $W_{T_0}, W_{T_1}, W_{T_2}, W_{T_3}, W_{T_4}$, where $W_{T_0} = 0$ by definition, yielding $\Delta W_i = W_{T_i} - W_{T_{i-1}}$.

By virtue of the Markov property of the gaussian random walk, the increments (differences of values at non-overlapping time intervals) of $W_t$ are uncorrelated. Therefore we have the values for $\Delta W_i$, where each $\Delta W_i$ is distributed like a Gaussian random variable of mean $0$ and variance $\sigma^2 T_i - T_{i-1}$. The therefore have the joint probability density function of these increments, which is the product of four ($i = 1,2,3,4$) pdf for Gaussian random variables. By substituting into the pdf the realized values of the random variables $\Delta W_i$, and leaving $\sigma$ and $T_1, T_2, T_3, T_4$ as parameters, we obtain the likelihood function $L(\sigma, T)$:

$$
L(\sigma, T) = \prod_{i=1}^4 \frac{1}{\sqrt{2 \pi \sigma^2 (T_i - T_{i-1})}} \exp \left(-\frac{{\Delta W_i}^2}{2 \sigma^2(T_i - T_{i-1})}\right)
$$

By maximizing the value of the likelihood function with respect to the parameters, we can find the parameters which are most likely to have generated the data. Actually there is an identifiability issue here, since $\sigma^2$ always multiplies the $T_i$, which means that if we divide $\sigma^2$ by a constant and multiply all of the $T_i$ by the same constant the value of the likelyhood stays the same. We can "fix" this issue by setting $T_1 - T_0 = 1$, and contenting ourselves to only learn the values up to a multiplicative constant.

In practice we have not only four values, of $\Delta W_1, \Delta W_2, \Delta W_3, \Delta W_4$, but we have four of them for every row. We can aggregate all the rows in a single likelihood, by assuming that their return performances are independent (might not be true), by taking the product of the likelihoods for each row, giving:

$$
L(\sigma_n, T) = \prod_{n \in rows} \prod_{i=1}^4 \frac{1}{\sqrt{2 \pi \sigma_n^2 (T_i - T_{i-1})}} \exp \left(-\frac{{\Delta W_{ni}}^2}{2 \sigma_n^2(T_i - T_{i-1})}\right)
$$

We let every row have a different volatility $\sigma_i$, but we can assume that across rows the time increments $T_1 - T_0  = 1$, $T_2 - T_1$, $T_3 - T_2$, $T_4 - T_3$ are constant.

Equivalently to maximizing the likelihood, we can minimize the negative log-likelihood
$$
\mathcal{l}(\sigma, T) = \sum_{n \in rows} \sum_{i=1}^4  \left(\frac{{\Delta W_{ni}}^2}{2 \sigma_n^2 (T_i - T_{i-1})} + \frac{1}{2}\log(T_i - T_{i-1}) + \log(\sigma_n)\right) + \text{const}
$$

While a closed form solution probably exists, we can optimize the log-likelihood numerically in little time, to obtain the estimated values of $\sigma_n$, the volatility for each trade, and $T_{i} - T_{i-1}$, the difference in time between `resp_i` and `resp_i-1`.

#### Assumptions:
Time horizons are the same across rows, returns follow random walk without drift, performances of different trades are independent.
These are all wrong with some degree of wrongness, but I hope the proposed model can still be useful.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# load as 32bits dtypes
types = pd.read_csv('../input/jane-street-market-prediction/train.csv', nrows=10).dtypes.astype(str).str.replace('64','32').to_dict()
data = pd.read_csv('../input/jane-street-market-prediction/train.csv', dtype = types)

resps = data.loc[:,['resp_1','resp_2','resp_3','resp_4']]

# get the log returns
W = np.log((resps + 1))

# get the increments
dW = W.diff(axis=1)
dW['resp_1'] = W['resp_1'] #the first increment is simply the first log return

In [ ]:
import torch
device = torch.device('cuda')

# initialize parameters

# 3 parameters for time differences (T_1 - T_0) is not a parameter as it is fixed to 1
dT_par = torch.tensor(np.arange(2,5), device=device, dtype=torch.float32)

# initialize the sigmas randomly
sigma = torch.tensor(np.random.randn(len(dW))*.01 + 1, device=device, dtype=torch.float)

# the actual parameters are the logarithms of the delta T and sigma, to enforce positivity and have
# more stable convergence
with torch.no_grad():
    dT_par_log = torch.log(dT_par).clone().requires_grad_(True)
    sigma_log = torch.log(sigma).clone().requires_grad_(True)

# load the data to gpu (increments)
dW = torch.tensor(dW.values, device=device)

In [ ]:
ITERS = 15000

# use Adam as it finds the right learning rates easily
opt = torch.optim.Adam([dT_par_log, sigma_log])

iteration = 0
while iteration < ITERS:
    # reset the gradients to 0
    opt.zero_grad() 
    
    # add the fixed 1 at the beginning of the time deltas
    dT_log = torch.cat([torch.zeros(1, device=device) , dT_par_log])
    
    # compute the log-likelihood (divided by number of rows)
    logL = 1/2 * torch.mean((dW**2  @ (1/torch.exp(dT_log))) * (1/torch.exp(2*sigma_log)))\
        + 1/2*torch.sum(dT_log)\
        + 4*torch.mean(sigma_log)
        
    # compute the gradient
    logL.backward()

    if iteration % 1000 == 0:
        with torch.no_grad():
            print(f'iter {iteration:8} {logL:.15} dT.grad {dT_par_log.grad.cpu().numpy()} sigma_log.grad.norm {torch.norm(sigma_log.grad):.9}')

    # execute one step of gradient descent
    opt.step()
    
    iteration+=1

# Results

# Sigma

As we saw earlier, the $\sigma_n$ variables are only identified up to a multiplicative constant, however the results show that the 50th percentile is at 0.3% volatility over the unit period, as given by our fixing of the first time delta to $1$, and the mean volatility is 0.5%. These seem like reasonable values for daily volatilities. If these were correct as they are, it would imply that our choice of the first time period as 1 day is correct.

In [ ]:
# let's bring stuff to CPU memory
sigmas, delta_times = torch.exp(sigma_log).detach().cpu().numpy(), torch.exp(dT_log).detach().cpu().numpy()

data['estimated_sigma'] = sigmas
data.estimated_sigma.describe()

## Correlation of sigma with features
We can also see that the $\sigma$ are significantly correlated with a few dataset features. which means that proxies of the volatilities, or the volatilities themselves were included in the dataset.

In [ ]:
corrs = data.loc[:,data.columns.str.startswith('feature')].corrwith(data['estimated_sigma'])

corrs.to_frame().style.background_gradient(cmap=plt.matplotlib.cm.Blues)

# Times
Summing back the time deltas gives us the estimated time horizons for  the returns in the `resp_i` columns.
`resp_1` and `resp_2` seem to be very close in time, with `resp_3` and `resp_4` spaced further apart. We are not really able to tell the meaning in days of these values. The choice of the first one as 1 day might be correct, however it makes the other ones look weird. Maybe all of them should be multiplied by 5, representing horizons over 1 week (5 trading days), 10 days (7 trading days), one month (~20 trading days), and three months (62 trading days). However the errors for the last one would be pretty substantial. This could be explained by the propagation of errors of our model further away in time.

In [ ]:
# if first delta is 1
np.cumsum(delta_times)

In [ ]:
# if first delta is 5 (as suggested by the second element being almost integer)
np.cumsum(delta_times)*5

# Conclusions

I hope you gained some insights from this notebook. In this exploration, we developed a method to estimate the time horizons of the `resp_i` features, and also estimate the variance of every trade. This method doesn't provide a target for training but it provides $\sigma$, which could be useful for risk management. It also provides some insight into the meaning of obfuscated features correlated with $\sigma$. The variable `resp` was kept out of the model because I didn't feel confortable assuming it would have a fixed time horizon for each trade, however we could find ways to test wether this is true and include it as well to gain more accuracy.

Many iterations of this work can, and probably will be repeated, by validating assumptions and refining the model.